## 1. Init


In [ ]:
from google.colab import drive
import os
import shutil
import subprocess

drive.mount('/content/drive')

CODE_FOLDER = 'code'
DATA_FOLDER = 'data'
PROJECT_DIR = '/content/drive/MyDrive/Text2SVG'
CODE_PATH = f"{PROJECT_DIR}/{CODE_FOLDER}"
DATA_PATH = f"{PROJECT_DIR}/{DATA_FOLDER}"


## 2. Clone Code Repository


In [ ]:
# Clean existing code directory and clone fresh repository
if os.path.exists(CODE_PATH):
    shutil.rmtree(CODE_PATH)

os.chdir(PROJECT_DIR)
result = subprocess.run(['git', 'clone', 'https://github.com/huanbasara/dual-branch-vae.git', CODE_FOLDER], 
                      capture_output=True, text=True)

if result.returncode != 0:
    raise Exception(f"Failed to clone repository: {result.stderr}")

print(f"✅ Code repository cloned successfully to {CODE_PATH}")


## 3. Install Dependencies

In [ ]:
%pip install torch torchvision numpy scipy pandas scikit-learn matplotlib pillow svglib svgpathtools
%pip install kornia opencv-python cairosvg pyyaml easydict tqdm

## 4. Test Environment Setup


In [ ]:
# Run comprehensive test suite
from utils.test_api import run_all_tests

# This will test:
# 1. Basic imports and system info
# 2. Project structure validation  
# 3. Model imports from Google Drive
# 4. Configuration file loading
success = run_all_tests()


In [ ]:
# Add code directory to Python path and change working directory
import sys
if CODE_PATH not in sys.path:
    sys.path.insert(0, CODE_PATH)
    
os.chdir(CODE_PATH)
print(f"✅ Working directory set to {CODE_PATH}")
